<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

Для выполнения проекта применять *BERT* необязательно.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem.wordnet import WordNetLemmatizer
import re 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,  make_scorer
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score

In [2]:
df= pd.read_csv('/datasets/toxic_comments.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Создадим функцию для лемматизации текста:

In [3]:
m = WordNetLemmatizer()

def lemmatize_text(text):    
    text = text.lower() # приведение к нижнему регистру
    lemm_text = "".join(m.lemmatize(text)) 
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) # очистка от лишних символов
    return " ".join(cleared_text.split())

df['text'] = df['text'].apply(lemmatize_text)

Из датафрема выделим целевой признак и обучающий набор данных. После разделим выборки на обучающие и тестовые 

In [4]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(features,target,test_size=0.4,random_state=142)


Уберем стоп слова:

In [5]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['text'].values)

features_test = count_tf_idf.transform(features_test['text'].values)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

Произведем обученине моделей. Для подбора параметров используем инструмент GridSearchCV.
Параметры моделей заранее подобраны таким образом, чтобы время обучения было минимальным при рассчитанной максимальной F1.

Начинаем с метода <b>Дерево решений</b>:

In [6]:
tree = DecisionTreeClassifier(random_state = 142)

grid_params = {
    'criterion':['gini', 'entropy'],
    'max_depth':list(range(15,16))
}

tree_gs = GridSearchCV(tree,param_grid=grid_params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)


print(f' Лучшая метрика F1 = {tree_gs.best_score_}  при параметрах {tree_gs.best_params_}')

Fitting 3 folds for each of 2 candidates, totalling 6 fits
 Лучшая метрика F1 = 0.620455223657525  при параметрах {'criterion': 'gini', 'max_depth': 15}


<b>Случайный Лес:</b>

In [7]:
forest = RandomForestClassifier(random_state = 142)

params = {
   'n_estimators':range(10,11),        
   'max_depth':list(range(15,16)) 
}

#pip_gs = make_pipeline(StandardScaler(), forest)
forest_gs = GridSearchCV(forest, param_grid=params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)

print(f' Лучшая метрика F1 = {forest_gs.best_score_}  при параметрах {forest_gs.best_params_}')


Fitting 3 folds for each of 1 candidates, totalling 3 fits
 Лучшая метрика F1 = 0.006436449765341505  при параметрах {'max_depth': 15, 'n_estimators': 10}


<b>Логистическая регрессия:</b>

In [8]:
lr = LogisticRegression(random_state = 142, class_weight='balanced')

params = {
   #'penalty':['l1','l2'],        
   'C':[15.0],
    'intercept_scaling' : range(15,16),
    'solver' : ['liblinear']
}

#pip_gs = make_pipeline(StandardScaler(), lr)
lr_gs = GridSearchCV(lr,param_grid=params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)

print(f' Лучшая метрика F1 = {lr_gs.best_score_}  при параметрах {lr_gs.best_params_}')



Fitting 3 folds for each of 1 candidates, totalling 3 fits
 Лучшая метрика F1 = 0.7635893989229517  при параметрах {'C': 15.0, 'intercept_scaling': 15, 'solver': 'liblinear'}


In [9]:
dt_predictions = tree_gs.predict(features_test)
print(f1_score(target_test, dt_predictions))


0.6274736943720436


In [10]:
dt_predictions = forest_gs.predict(features_test)
print(f1_score(target_test, dt_predictions))

0.0009021199819576004


In [11]:
dt_predictions = lr_gs.predict(features_test)
print(f1_score(target_test, dt_predictions))

0.767351069982649


## Выводы

В ходе проекта были обработаны данные для интернет-магазина. В рамках запуска нового сервиса с возможностью оставлять комментарии пользователями необходимо производить поиск токсичных комментариев. Чтобы классифицировать комментарии на позитивные и негативные были построены 3 модели
Самой эффективной моделью, которая показала максимальную метрику F1(0.76) является модель Логистической регресии. Требование о значении метрики F1 (не менее 0,75) выполнено.